In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# MLP from sklearn
from sklearn.neural_network import MLPClassifier


# MNIST 
X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
# Preprocessing normalisation 
X_ = X_ / 255

y_ = y_.astype(int)

# On-hot encoding
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y_.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)


m:\ML\ML_regs\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [61]:
sk_mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
                          learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True,
                            random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
                                nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                                    beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

In [45]:
# Train 
sk_mlp.fit(X_train, y_train)

# Predict
y_pred = sk_mlp.predict(X_test)




Accuracy:  9.9585


In [57]:
# Accuracy
accuracy = np.sum(y_pred == y_test) / len(y_test)
print("Accuracy: ", accuracy)

Accuracy:  9.9585


In [63]:
model = MultiPreClf.MultilayerPerceptron(alpha=0.0001, 
                                         n_iterations=5, 
                                         input_layer_size=784, 
                                         hidden_layers_size=[128, 64], 
                                         n_outputs=10, 
                                         regularization="l2", 
                                         lambda_reg=0.1)


In [64]:
training_loss, validation_loss = model.train(X_train, y_train, X_test, y_test)


Iteration: 0
Epoch: 0, Training loss : 2.3016695865388983
Epoch: 0, valid loss: 2.3015143449479103
Iteration: 1
Epoch: 1, Training loss : 2.301362243589438
Epoch: 1, valid loss: 2.3011120200454025
Iteration: 2
Epoch: 2, Training loss : 2.301267823288329
Epoch: 2, valid loss: 2.3009650463508216
Iteration: 3
Epoch: 3, Training loss : 2.3012381027982873
Epoch: 3, valid loss: 2.3009194540727544
Iteration: 4
Epoch: 4, Training loss : 2.30122775913255
Epoch: 4, valid loss: 2.3009008628468357


In [34]:
# Step 3: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Step 4: Calculate Accuracy
accuracy = model.accuracy(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 11.43%


In [37]:
y_pred

array([[0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       ...,
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438]])

In [35]:
y_pred_classes

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [36]:
y_test_classes

array([8, 4, 8, ..., 2, 7, 1], dtype=int64)